In [ ]:
# Use sourmash to make kmers
cd path/to/trimmed/and/cleaned/reads
mkdir ../out/signatures
for f in *R1*
do
sample=${f%%_S*}
sourmash compute \
-k 21,31,51 \
--scaled 1000 \
--track-abundance \
-o ../out/signatures/${sample}.sig \
--merge ${sample} \
./${sample}*
done

# Then run abundance parser
cd path/to/out/signatures 
for f in *.sig
do
sample=${f%%.sig}
cut -f2 -d ] ${sample}.sig | cut -f2 -d [ > ${sample}.21.abs.csv
cut -f4 -d ] ${sample}.sig | cut -f2 -d [ > ${sample}.31.abs.csv
cut -f6 -d ] ${sample}.sig | cut -f2 -d [ > ${sample}.51.abs.csv
done

# Then aggregate for the 31mers
for f in *.sig
do
sample=${f%%.sig}
tr ',' '\n' < ${sample}.31.abs.csv | \
sort | uniq -c | awk '{print $2","$1}' | \
sed 's/$/,k31/' > ${sample}.31.freq
done

# Lastly, we use R to make complexity object
# Specifically, we will use make_complexity_obj.Rmd